In [1]:
PATH = "../datasets/wiki/"
import os
datatypes = os.listdir(PATH)

def get_contents(file_path):
    sentences = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            sentence = line.strip()
            sentences.append(sentence)
    return sentences

def get_duals(datatypes,lang):
    for i in datatypes:
        x = os.listdir(os.path.join(PATH,i))
        x.remove("domain.txt")
        if lang in x[0] or lang in x[1]:
            res_lang = get_contents(os.path.join(PATH,i,x[0]))
            res_en = get_contents(os.path.join(PATH,i,"train.eng_Latn"))
            return {lang:res_lang,"en":res_en}
        

In [3]:
def get_contents(file_path):
    sentences = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            sentence = line.strip()
            sentences.append(sentence)
    return sentences

In [4]:
def get_duals(datatypes,lang):
    for i in datatypes:
        x = os.listdir(os.path.join(PATH,i))
        x.remove("domain.txt")
        if lang in x[0] or lang in x[1]:
            res_lang = get_contents(os.path.join(PATH,i,x[0]))
            res_en = get_contents(os.path.join(PATH,i,"train.eng_Latn"))
            return {lang:res_lang,"en":res_en}
        

In [5]:
data = get_duals(datatypes,"hi")

In [1]:
from transformers import XLNetTokenizer, XLNetLMHeadModel, AdamW, get_linear_schedule_with_warmup

model = XLNetLMHeadModel.from_pretrained('xlnet-base-cased')
model.num_parameters()

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


116750336

In [6]:
import pandas as pd
import numpy as np

pd.DataFrame(data)

,hi,en
0,"""संयुक्त प्रतिस्पर्धा"" वा ""बहु प्रतिस्पर्धा "" ...","There are also ""combined events"" or ""multi eve..."
1,"आवेदन जमा करबाक बाद, अहाँकेँ किछु व्यावसायिक द...","After submitting your application, you should ..."
2,१९९० के दशक मे दमानिया शिपिंग द्वारा संचालित म...,There was also a short-lived catamaran service...
3,"एहन रोगीक लेल, जनिक संक्रमण आ कैंसरक नैदानिक स...",X-rays are a less costly initial option offere...
4,"एलर्जी सँ पीड़ित नेत्रश्लेष्माशोथक लेल, मुह के...","For allergic conjunctivitis, cool water poured..."
...,...,...
40265,भारत सरकार ने उनके निधन पर एक दिन के राष्ट्रीय...,The Government of India declared a day of nati...
40266,"उनकी बेटी शांति मेनन हैं, जो बैंगलोर में एक स्...","His daughter is Shanti Menon, who is the princ..."
40267,वर्ष 1975 में विश्व बैंक की एक तकनीकी टीम ने '...,"In the year 1975, a technical team from the Wo..."
40268,उनके पार्थिव शरीर को शहनाई के साथ पुराने वाराण...,His body along with a Shehnai was buried at Fa...


In [8]:
stre = "a proposal has been made."
len(stre)
english_vocabulary = ["START_TOKEN", ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', 
                        '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
                        ':', '<', '=', '>', '?', '@',
                        '[', '\\', ']', '^', '_', '`', 
                        'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l',
                        'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 
                        'y', 'z', 
                        '{', '|', '}', '~', "PADDING_TOKEN", "END_TOKEN"]
english_vocabulary[15]

'.'

In [2]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

model_name = "t5-small"  # You can choose a different variant like t5-base, t5-large, etc.
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

model.num_parameters()

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


60506624

In [9]:
from sklearn.model_selection import train_test_split

input_texts = data['hi']
target_texts = data['en']

train_input_texts, test_input_texts, train_target_texts, test_target_texts = train_test_split(input_texts, target_texts, test_size=0.2, random_state=42)


In [10]:
import torch
from torch.utils.data import Dataset, DataLoader

class TranslationDataset(Dataset):
    def __init__(self, input_texts, target_texts, tokenizer, max_length=128):
        self.input_texts = input_texts
        self.target_texts = target_texts
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.input_texts)

    def __getitem__(self, idx):
        input_text = "translate Hindi to English: " + self.input_texts[idx]
        target_text = self.target_texts[idx]

        input_encodings = self.tokenizer(input_text, padding='max_length', truncation=True, max_length=self.max_length, return_tensors="pt")
        target_encodings = self.tokenizer(target_text, padding='max_length', truncation=True, max_length=self.max_length, return_tensors="pt")

        input_ids = input_encodings.input_ids.squeeze()
        attention_mask = input_encodings.attention_mask.squeeze()
        labels = target_encodings.input_ids.squeeze()
        labels[labels == self.tokenizer.pad_token_id] = -100  # Replace padding token id's with -100 to ignore padding tokens in loss

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': labels
        }

# Initialize tokenizer and datasets
from transformers import T5Tokenizer

tokenizer = T5Tokenizer.from_pretrained("t5-small")

train_dataset = TranslationDataset(train_input_texts, train_target_texts, tokenizer)
test_dataset = TranslationDataset(test_input_texts, test_target_texts, tokenizer)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [11]:
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=8)

In [ ]:
from transformers import T5ForConditionalGeneration, AdamW, get_scheduler

model = T5ForConditionalGeneration.from_pretrained("t5-small")

optimizer = AdamW(model.parameters(), lr=5e-5)
num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

# Training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model.to(device)

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

    print(f"Epoch {epoch+1} finished")


/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


cuda


In [ ]:
import sacrebleu
model.eval()
predictions = []
references = []

with torch.no_grad():
    for batch in test_dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels']

        generated_ids = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=50)
        preds = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
        targets = tokenizer.batch_decode(labels, skip_special_tokens=True)

        predictions.extend(preds)
        references.extend(targets)

# Calculate BLEU score
bleu = sacrebleu.corpus_bleu(predictions, [references])
print(f"BLEU score: {bleu.score}")


In [ ]:
model.eval()
input_text = "translate Hindi to English: आपका नाम क्या है?"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids

with torch.no_grad():
    generated_ids = model.generate(input_ids, max_length=50)
    translation = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

print(translation)
